# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [1]:
# import math

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn import metrics
# from sklearn.ensemble import RandomForestClassifier

# from scipy.stats import ks_2samp
import statsmodels.formula.api as smf
import statsmodels.api as sm
import patsy

%matplotlib inline


from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score

In [2]:
previsao_de_renda = 'https://raw.githubusercontent.com/Jefersonfranca/EBAC_Curso_Cientista_de_Dados/main/M%C3%B3dulo%2013%20Regress%C3%A3o%20II/database/previsao_de_renda.csv'

df = pd.read_csv(previsao_de_renda, index_col=0)
df

,data_ref,id_cliente,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,2015-01-01,15056,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,2015-01-01,9968,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,2015-01-01,4312,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,2015-01-01,10639,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,2015-01-01,7064,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,2016-03-01,16006,F,False,True,0,Empresário,Secundário,Solteiro,Casa,48,13.887671,1.0,7990.58
14996,2016-03-01,3722,F,False,True,0,Pensionista,Superior completo,Solteiro,Casa,57,NaN,1.0,10093.45
14997,2016-03-01,6194,F,True,True,0,Assalariado,Superior completo,Casado,Casa,45,7.832877,2.0,604.82
14998,2016-03-01,4922,M,True,False,0,Empresário,Superior completo,Casado,Casa,36,4.298630,2.0,3352.27


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15000 entries, 0 to 14999
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   data_ref               15000 non-null  object 
 1   id_cliente             15000 non-null  int64  
 2   sexo                   15000 non-null  object 
 3   posse_de_veiculo       15000 non-null  bool   
 4   posse_de_imovel        15000 non-null  bool   
 5   qtd_filhos             15000 non-null  int64  
 6   tipo_renda             15000 non-null  object 
 7   educacao               15000 non-null  object 
 8   estado_civil           15000 non-null  object 
 9   tipo_residencia        15000 non-null  object 
 10  idade                  15000 non-null  int64  
 11  tempo_emprego          12427 non-null  float64
 12  qt_pessoas_residencia  15000 non-null  float64
 13  renda                  15000 non-null  float64
dtypes: bool(2), float64(3), int64(3), object(6)
memory usage: 1

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

#### 1) Seprando a base em treinamento e teste

In [4]:
df_train, df_test = train_test_split(df, test_size=0.25, random_state=42)

modelo ='''renda ~ sexo + posse_de_veiculo 
        + posse_de_imovel 
        + qtd_filhos 
        + tipo_renda 
        + educacao 
        + estado_civil 
        + tipo_residencia 
        + idade 
        + tempo_emprego 
        + qt_pessoas_residencia'''

y_train, X_train = patsy.dmatrices(formula_like=modelo, data=df_train)
y_test, X_test = patsy.dmatrices(formula_like=modelo, data=df_test)

#### 2) Rodando uma regularização Ridge com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1].

In [5]:
#Regularização ridge
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
r2_y_pred = []

for alpha in alphas:
    md = sm.OLS(y_train, X_train)
    reg = md.fit_regularized(method='elastic_net', 
                             refit=True, 
                             L1_wt=0,  # ridge fit
                             alpha=alpha)
    y_pred = reg.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    
    r2_y_pred.append(r2)
    print(f'Alpha {alpha}: \nR-squared = {r2}\n')
    
print(pd.DataFrame({'alpha':alphas, 
              '𝑅2':r2_y_pred
             }).sort_values(by='𝑅2', ascending=False))

Alpha 0: 
R-squared = 0.26243403442818514

Alpha 0.001: 
R-squared = 0.26226194952350634

Alpha 0.005: 
R-squared = 0.2619663776008281

Alpha 0.01: 
R-squared = 0.2615684758582858

Alpha 0.05: 
R-squared = 0.25638277607326476

Alpha 0.1: 
R-squared = 0.24863408637050965

   alpha        𝑅2
0  0.000  0.262434
1  0.001  0.262262
2  0.005  0.261966
3  0.010  0.261568
4  0.050  0.256383
5  0.100  0.248634


###### Entre os modelos de regressão com regularização ridge testados, o que obteve o melhor coeficiente de determinação foi aquele que utilizou um valor de alpha igual a 0.000, resultando em um 𝑅2 de 0.262434.

#### 3) Regressão LASSO com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1]. 


In [6]:
# Regularização Lasso
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
r2_y_pred = []

for alpha in alphas:
    md = sm.OLS(y_train, X_train)
    reg = md.fit_regularized(method='elastic_net', 
                                 refit=True, 
                                 L1_wt=1,  # lasso fit
                                 alpha=alpha)
       
    r2 = reg.rsquared
    r2_y_pred.append(r2)    
    
    print(f'Alpha {alpha}: \nR-squared = {round(r2*100,3)}\n')
    
print(pd.DataFrame({'alpha':alphas, 
              '𝑅2':r2_y_pred
             }).sort_values(by='𝑅2', ascending=False))
reg.summary()

Alpha 0: 
R-squared = 25.377

Alpha 0.001: 
R-squared = 25.377

Alpha 0.005: 
R-squared = 25.377

Alpha 0.01: 
R-squared = 25.377

Alpha 0.05: 
R-squared = 25.376

Alpha 0.1: 
R-squared = 25.377

   alpha        𝑅2
0  0.000  0.253775
1  0.001  0.253775
2  0.005  0.253775
3  0.010  0.253775
5  0.100  0.253775
4  0.050  0.253762


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.254
Model:                            OLS   Adj. R-squared:                  0.252
Method:                 Least Squares   F-statistic:                     126.3
Date:                Thu, 30 May 2024   Prob (F-statistic):               0.00
Time:                        11:45:14   Log-Likelihood:                -96560.
No. Observations:                9313   AIC:                         1.932e+05
Df Residuals:                    9288   BIC:                         1.934e+05
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                        -5598.6621   2891.222     -1.936      0.053   -1.13e+04      68.767
sexo[T.M]                         6008.2148    181.801     33.048      0.000    5651.846    6364.584
posse_de_veiculo[T.True]            58.5213    175.147      0.334      0.738    -284.804     401.847
posse_de_imovel[T.True]            270.2747    172.590      1.566      0.117     -68.039     608.588
tipo_renda[T.Bolsista]           -1105.6608   3156.093     -0.350      0.726   -7292.295    5080.973
tipo_renda[T.Empresário]           876.9981    184.118      4.763      0.000     516.086    1237.910
tipo_renda[T.Pensionista]        -1779.9725   3862.869     -0.461      0.645   -9352.044    5792.099
tipo_renda[T.Servidor público]      98.5600    279.138      0.353      0.724    -448.612     645.732
educacao[T.Pós graduação]         1698.0431   2067.763      0.821      0.412   -2355.225    5751.312
educacao[T.Secundário]             684.1460    875.249      0.782      0.434   -1031.535    2399.827
educacao[T.Superior completo]     1307.2415    878.456      1.488      0.137    -414.724    3029.207
educacao[T.Superior incompleto]    153.7895    950.342      0.162      0.871   -1709.089    2016.668
estado_civil[T.Separado]          1158.0317   1367.598      0.847      0.397   -1522.761    3838.824
estado_civil[T.Solteiro]           733.9159   1338.428      0.548      0.583   -1889.697    3357.529
estado_civil[T.União]             -304.1734    311.999     -0.975      0.330    -915.761     307.414
estado_civil[T.Viúvo]             1207.4006   1418.351      0.851      0.395   -1572.879    3987.680
tipo_residencia[T.Casa]           -204.1053    688.648     -0.296      0.767   -1554.006    1145.796
tipo_residencia[T.Com os pais]    -238.7275    765.441     -0.312      0.755   -1739.161    1261.706
tipo_residencia[T.Comunitário]    -290.8336   1276.270     -0.228      0.820   -2792.602    2210.935
tipo_residencia[T.Estúdio]         811.9680   1210.109      0.671      0.502   -1560.111    3184.048
tipo_residencia[T.Governamental]   328.4661    827.311      0.397      0.691   -1293.245    1950.177
qtd_filhos                       -1185.5366   1327.742     -0.893      0.372   -3788.202    1417.129
idade                               37.1551      9.816      3.785      0.000      17.914      56.397
tempo_emprego                      552.5928     12.750     43.341      0.000     527.600     577.585
qt_pessoas_residencia             1286.8942   1324.943      0.971      0.331   -1310.284    3884.073
==============================================================================
Omnibus:                    13488.001   Durbin-Watson:                   2.017
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          8605851.129
Skew:             

#### Um insight importante da análise dos modelos de regressão com regularização Lasso testado é que não houve mudanças consideráveis nos resultados. Os valores de R² ficaram muito próximos de 25,38%, indicando uma consistência no desempenho dos modelos independente do alpha utilizado.

##### 4) Rode um modelo stepwise e avaliando o  𝑅²  na base de testes. 

In [7]:
df_dummies = pd.get_dummies(data=df.dropna(), drop_first=True).astype(int)

X = df_dummies.drop(columns='renda')
y = df_dummies['renda']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [8]:
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.05, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded, dtype=np.dtype('float64'))
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.index[new_pval.argmin()]
            included.append(best_feature)
            changed=True
            if verbose:
                 print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        print("#############")
        print(included)
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

variaveis = stepwise_selection(X_test, y_test)

print('\n\nresulting features:')
print(variaveis)

Add  tempo_emprego                  with p-value 1.15416e-120
#############
['tempo_emprego']
Add  sexo_M                         with p-value 1.97372e-116
#############
['tempo_emprego', 'sexo_M']
Add  educacao_Superior completo     with p-value 0.00030172
#############
['tempo_emprego', 'sexo_M', 'educacao_Superior completo']
Add  idade                          with p-value 0.000294673
#############
['tempo_emprego', 'sexo_M', 'educacao_Superior completo', 'idade']
Add  posse_de_veiculo               with p-value 0.00268263
#############
['tempo_emprego', 'sexo_M', 'educacao_Superior completo', 'idade', 'posse_de_veiculo']
Add  tipo_renda_Empresário          with p-value 0.0127891
#############
['tempo_emprego', 'sexo_M', 'educacao_Superior completo', 'idade', 'posse_de_veiculo', 'tipo_renda_Empresário']
Add  qt_pessoas_residencia          with p-value 0.0167073
#############
['tempo_emprego', 'sexo_M', 'educacao_Superior completo', 'idade', 'posse_de_veiculo', 'tipo_renda_Empresário

In [9]:
X = df_dummies[variaveis]
y = df_dummies['renda']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

y_pred = sm.OLS(y_train, X_train).fit().predict(X_test)
print(f'R-squared: {r2_score(y_test, y_pred)}')

R-squared: 0.2877077014713394


In [10]:
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
r2_y_pred = []

for alpha in alphas:
    md = sm.OLS(y_train, X_train)
    reg = md.fit_regularized(method='elastic_net', 
                             refit=True, 
                             L1_wt=0,  # ridge fit
                             alpha=alpha)
    y_pred = reg.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    
    r2_y_pred.append(r2)
    print(f'Alpha {alpha}: \nR-squared = {r2}\n')
    
print(pd.DataFrame({'alpha':alphas, 
              '𝑅2':r2_y_pred
             }).sort_values(by='𝑅2', ascending=False))

Alpha 0: 
R-squared = 0.2877077014713392

Alpha 0.001: 
R-squared = 0.2879367976026064

Alpha 0.005: 
R-squared = 0.2886866743929598

Alpha 0.01: 
R-squared = 0.2893704400522459

Alpha 0.05: 
R-squared = 0.289642839977477

Alpha 0.1: 
R-squared = 0.28449206638537794

   alpha        𝑅2
4  0.050  0.289643
3  0.010  0.289370
2  0.005  0.288687
1  0.001  0.287937
0  0.000  0.287708
5  0.100  0.284492


In [11]:
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
r2_y_pred = []

for alpha in alphas:
    md = sm.OLS(y_train, X_train)
    reg = md.fit_regularized(method='elastic_net', 
                             refit=True, 
                             L1_wt=1,  # lasso fit
                             alpha=alpha)
    y_pred = reg.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    
    r2_y_pred.append(r2)
    print(f'Alpha {alpha}: \nR-squared = {r2}\n')
    
print(pd.DataFrame({'alpha':alphas, 
              '𝑅2':r2_y_pred
             }).sort_values(by='𝑅2', ascending=False))
reg.summary()

Alpha 0: 
R-squared = 0.2877077014713394

Alpha 0.001: 
R-squared = 0.2877077014713394

Alpha 0.005: 
R-squared = 0.2877077014713394

Alpha 0.01: 
R-squared = 0.2877077014713394

Alpha 0.05: 
R-squared = 0.2877077014713394

Alpha 0.1: 
R-squared = 0.2877077014713394

   alpha        𝑅2
0  0.000  0.287708
1  0.001  0.287708
2  0.005  0.287708
3  0.010  0.287708
4  0.050  0.287708
5  0.100  0.287708


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  renda   R-squared (uncentered):                -328.309
Model:                            OLS   Adj. R-squared (uncentered):           -328.627
Method:                 Least Squares   F-statistic:                             -1031.
Date:                Thu, 30 May 2024   Prob (F-statistic):                        1.00
Time:                        11:45:34   Log-Likelihood:                         -97095.
No. Observations:                9320   AIC:                                  1.942e+05
Df Residuals:                    9311   BIC:                                  1.943e+05
Df Model:                           9                                                  
Covariance Type:            nonrobust                                                  
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
tempo_emprego                569.5096     13.215     43.094      0.000     543.604     595.415
sexo_M                      6025.3062    185.368     32.505      0.000    5661.945    6388.667
educacao_Superior completo   412.5429    171.834      2.401      0.016      75.711     749.375
idade                         -1.2933      5.236     -0.247      0.805     -11.556       8.970
posse_de_veiculo            -187.3780    180.702     -1.037      0.300    -541.594     166.838
tipo_renda_Empresário        685.3807    186.966      3.666      0.000     318.887    1051.875
qt_pessoas_residencia       -152.7325     76.790     -1.989      0.047    -303.257      -2.208
data_ref_2015-06-01          766.6469    336.452      2.279      0.023     107.128    1426.166
tipo_residencia_Estúdio     -356.4364   1059.844     -0.336      0.737   -2433.963    1721.090
==============================================================================
Omnibus:                    13633.206   Durbin-Watson:                   1.995
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          8218628.878
Skew:                           8.669   Prob(JB):                         0.00
Kurtosis:                     147.441   Cond. No.                         538.
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### Na minha opinião, o melhor resultado dos modelos stepwise foi o que utilizou a regressão LASSO, pois obteve desempenho semelhante em todos os valores de alpha e ainda assim alcançou um coeficiente de determinação comparável ao modelo que utilizou a regularização ridge com o maior valor desse coeficiente. Dessa forma, esse modelo pode ser considerado mais simples e, ao mesmo tempo, apresentar o melhor resultado entre as opções desenvolvidas.

### 5) Qual modelo você acha o melhor de todos?

A melhor opção dentre os modelos mencionados é o modelo de regressão Lasso com seleção de variáveis do tipo stepwise. Mesmo utilizando um número menor de variáveis em comparação aos outros modelos, este modelo foi capaz de obter um coeficiente de determinação muito próximo ou maior, apresentando pouca variação nos resultados entre os parâmetros.

### 6) Tentando melhorar a base de testes dos modelos usados.

In [12]:
df_matrix = df_dummies.loc[:, variaveis + ['renda']]
df_matrix.rename(columns={'educacao_Superior completo': 'educacao_Superior_completo'}, 
                 inplace=True)

df_train, df_test = train_test_split(df_matrix, test_size=0.25, random_state=42)

modelo = smf.ols(formula='''
                         np.log(renda) ~ tempo_emprego 
                                         + sexo_M 
                                         + educacao_Superior_completo
                                         + idade 
                                         - posse_de_veiculo 
                                         + tipo_renda_Empresário 
                                         + qt_pessoas_residencia
                                         - tipo_residencia_Estúdio
                                         ''', data=df_train)

reg = modelo.fit_regularized(method='elastic_net', 
                             refit=True, 
                             L1_wt=1,
                             alpha=0)

print(reg.summary())

y_pred = reg.predict(df_test)
print('\nCoeficiente de determinação (𝑅2) na base de testes:', 
      r2_score(df_test['renda'], 
               np.exp(y_pred)
              )
     )

                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.348
Model:                            OLS   Adj. R-squared:                  0.347
Method:                 Least Squares   F-statistic:                     709.8
Date:                Thu, 30 May 2024   Prob (F-statistic):               0.00
Time:                        11:45:53   Log-Likelihood:                -10235.
No. Observations:                9320   AIC:                         2.049e+04
Df Residuals:                    9313   BIC:                         2.054e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [13]:
X = df_dummies[variaveis]
y = df_dummies['renda']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

reg_tree = DecisionTreeRegressor(random_state=42, max_depth=5, min_samples_leaf=13)
reg_tree.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=5, min_samples_leaf=13, random_state=42)

In [14]:
print('Coeficiente de determinação (𝑅²) na base de testes:', round(reg_tree.score(X=X_test, y=y_test)*100, 2), '%')

Coeficiente de determinação (𝑅²) na base de testes: 38.91 %
